# 💻 Raster

<div class="alert alert-info">

**Ekstra biblioteker**

Senere i denne notebooken skal vi bruke bibliotekene `xarray` og `rioxarray`, det kan være en god ide å installere de allerede nå, så det er klart til bruk senere:

`conda install -c conda-forge xarray rioxarray`

</div>

Rasterdata er mye brukt til å representere og lagre data om kontinuerlige overflater, der hver piksel inneholder spesifikk informasjon (karakteristikk, utstråling, spektralsignaturer) om et spesifikt område av jorden, for eksempel et 10x10 meter område. Denne notebooken gir en introduksjon til behandling av rasterdata i Python, og er basert på [kapittel 7 i Python for Geographic Data Analysis](https://pythongis.org/part2/chapter-07/index.html) 

Det er forskjellige grunner til at du kanskje vil lagre dataene dine i rasterformat:

- Det er en enkel datastruktur: En matrise av celler med verdier som representerer informasjon om den observerte overflaten/fenomenene
- Det er en effektiv måte å lagre data fra store sammenhengende flater
- Det er et kraftig format som kan brukes til ulike romlige og statistiske analyser
- Du kan utføre raske overlay med flere lag

## Arbeide med rasterdata i Python

Det er en rekke biblioteker som er mye brukt når du arbeider med rasterdata i Python:

- `xarray` gir en brukervennlig og intuitiv måte å jobbe med flerdimensjonale rasterdata med koordinater og attributter (noe lik `geopandas` som brukes til vektordatabehandling),
- `rioxarray` gir metoder for å utføre GIS-relaterte operasjoner med rasterdata (f.eks. lesing/skriving, reprojisering, klipping, resampling),
- "xarray-spatial" gir metoder for å analysere rasterdata (f.eks. fokale/sonale operasjoner, overflateanalyse, banefinning),
- `rasterio` kjernebibliotek for arbeid med GIS-rasterdata. `rioxarray` er en utvidelse av dette biblioteket som bringer de samme funksjonalitetene på toppen av `xarray`-biblioteket,
- `numpy` er et kjerne-bibliotek i Python for numerisk databehandling som brukes til å representere og arbeide med flerdimensjonale arrays. `numpy` har stor innflytelse på hvordan de andre rasterbibliotekene fungerer og kan brukes til å generere flerdimensjonale arrays fra bunnen av.
- 
## Lage et enkelt rasterlag ved å bruke `numpy`

For å få en bedre følelse av hvordan rasterdataene ser ut, kan vi starte med å lage en enkel todimensjonal array i Python ved å bruke `numpy`. I det følgende vil vi modifisere rasterlaget til å representere et enkelt terreng som har en høyde midt i rutenettet. Vi gjør dette ved å sette høyere verdier i midten mens de andre verdiene er representert med verdi 0. La oss starte med å importere `numpy` og `matplotlib`-biblioteket som vi bruker for å visualisere dataene våre:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

Å lage et enkelt 2D-rasterlag (en 2D-matrise) med 10x10-matrise kan enkelt gjøres ved å bruke en `numpy`-metode `.zeros()` med fyller cellene (piksler) med nuller. Hver null representerer en standard pikselverdi (f.eks. 0 høyde). Du kan tenke på dette som et tomt rasternett:

Nå har vi en enkel 2D-matrise fylt med nuller. Deretter modifiserer vi rasterlaget for å representere et enkelt terreng og legger til større tall i midten av rutenettet ved å sette høyere verdier i midten. Vi kan gjøre dette ved å *{term}`slice`* numpy-matrisen ved å bruke indeksene til matrisen og oppdatere tallene på disse stedene til å være høyere. Å skjære "numpy"-matriser skjer på lignende måte som når du arbeider med Python-lister og får tilgang til elementene i en liste (se kapittel 2.2). Men i dette tilfellet gjør vi dette i to dimensjoner ved å få tilgang til verdiene som er lagret i spesifikke rader og kolonner ved å følge syntaksen: `[start-rad-idx: end-row-idx, start-col-idx: end-col- idx]`. Dermed kan vi oppdatere verdiene i vår 2D-matrise som følger:

Her oppdaterte vi først cellene mellom den fjerde og syvende raden og kolonnen (`[4:7, 4:7]`) til å ha en verdi `5`, og deretter oppdaterte vi midten av matrisen til å representere toppen av bakken med verdien `10`. Som en Resultatet har vi et enkelt rasterlag som simulerer et enkelt terreng. Vi kan også plotte dette rasterlaget ved å bruke `matplotlib`-biblioteket og dets `.imshow()`-funksjon som kan brukes til å visualisere arrays:

# Introduksjon til datastrukturer i xarray

Nå som du har lært litt grunnleggende om rasterdata og hvordan du lager en enkel 2-dimensjonal raster-array ved å bruke "numpy", fortsetter vi å utforske på en mer omfattende måte hvordan du arbeider med virkelige rasterdata ved å bruke "xarray" og `rioxarray`-biblioteker (+ andre relevante biblioteker knyttet til dem). `xarray`-biblioteket er et svært nyttig verktøy for å lagre, representere og manipulere rasterdata, mens `rioxarray` gir forskjellige rasterbehandlingsfunksjoner (GIS) på toppen av `xarray`-datastrukturene, for eksempel lesing og skriving av flere forskjellige rasterformater og utføre ulike geoberegningsoppgaver. Under panseret bruker `rioxarray` et annet Python-bibliotek kalt `rasterio` (som fungerer med N-dimensjonale `numpy`-matriser), men fordelen med `xarray` og `rioxarray` er at de gir en enklere og mer intuitiv måte å jobbe med rasterdatalag, på en litt lignende måte som å jobbe med vektordata ved å bruke `geopandas`.

Når du arbeider med rasterdata har du vanligvis ulike lag som representerer ulike geografiske trekk ved verden (f.eks. høyde, temperatur, nedbør osv.), og disse dataene blir muligens fanget opp på ulike tider av året/dagen/timen, noe som betyr at du har longitudinelle observasjoner fra samme område, som utgjør tidsseriedata. Oftere enn ikke må du kombinere informasjon fra disse lagene for å kunne gjennomføre meningsfulle analyser basert på dataene, for eksempel å lage en værmelding. En av de største fordelene med "xarray" er at du enkelt kan lagre, kombinere og analysere alle disse forskjellige lagene via et enkelt objekt, det vil si et "Dataset", som vist i figur 7.2.

De to grunnleggende datastrukturene som tilbys av 'xarray'-biblioteket er 'DataArray' og 'Dataset' (Figur 7.2). Begge bygger på og utvider styrkene til bibliotekene "numpy" og "pandas". 'DataArray' er en merket N-dimensjonal array som ligner på 'pandas.Series' men fungerer med rasterdata (lagret som 'numpy' arrays). "Datasettet" er igjen en flerdimensjonal array-database i minnet som inneholder flere "DataArray"-objekter. I tillegg til variablene som inneholder observasjonene av et gitt fenomen, har du også `x`- og `y`-koordinatene til observasjonene lagret i separate lag, samt metadata som gir relevant informasjon om dataene dine, for eksempel Koordinatreferansesystem og /eller tid. Dermed er et "datasett" som inneholder rasterdata veldig likt "geopandas.GeoDataFrame", og faktisk kan forskjellige "xarray"-operasjoner føles veldig kjent hvis du har lært det grunnleggende om "pandas" og "geopandas" dekket i kapittel 3 og 6.


![***Figure 7.2.** Key `xarray` data structures. Image source: Xarray Community (2024), lisensiert under Apache 2.0.*](https://docs.xarray.dev/en/stable/_images/dataset-diagram.png)

***Figur 7.2** Nøkkel "xarray" datastrukturer. Bildekilde: [Xarray Community](https://tutorial.xarray.dev/fundamentals/01_data_structures.html) (2024), lisensiert under Apache 2.0.*

Noen av fordelene med "xarray" inkluderer:

- Et mer intuitivt og brukervennlig grensesnitt for å jobbe med flerdimensjonale arrays (sammenlignet f.eks. med "numpy")
- Muligheten til å velge og kombinere data langs en dimensjon på tvers av alle arrays i et "Datasett" samtidig
- Kompatibilitet med et stort økosystem av Python-biblioteker som fungerer med arrays/rasterdata
- Tett integrering av funksjonaliteter fra velkjente Python-dataanalysebiblioteker, som "pandas", "numpy", "matplotlib" og "dask".

## Lese inn en fil

Vi skal nå bruke `xarray` og `rioaxarray` til å lese inn rasterfilen vår. `xarray` og `rioaxarray` er ikke en del av oppsettet vi satte opp i `conda`-miljøet i starten av kurset. Hvis du ikke har installert det på egenhånd, kan det derfor være nødvendig å kjøre `conda install -c conda-forge xarray rioxarray` for å sørge for at den neste delen av notebooken fungerer.


In [ ]:
import xarray as xr
import rasterio
from osgeo import gdal
import pathlib
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_MAPPE = NOTEBOOK_PATH / "data"

import rioxarray

url = DATA_MAPPE/"Aas_20240625.tif"


Nå har vi lest GeoTIFF-filen inn i en `xarray.Dataset` datastruktur som vi lagret i en variabel `data`. "Datasettet" inneholder de faktiske dataverdiene for rastercellene, samt annen relevant attributtinformasjon relatert til dataene

In [ ]:
# Koordinatene i datasettet


In [ ]:
# Projeksjonen til datasettet


In [ ]:
# Variabler i datasettet


In [ ]:
# Hvor mange og hvilke bånd


### NDVI

En vanlig operasjon er å beregne `NDVI` (Normalized difference vegetation index), noe som ganske enkelt lar seg gjøre:

In [ ]:
NiR=data['band_data'][3,:,:]#Band 4 as counting starts from 0
Red=data['band_data'][2,:,:]#Band 3 as counting starts from 0
NDVI=(NiR-Red)/(NiR+Red)

In [ ]:
NDVI

In [ ]:
print('The variable type of the data:',type(NDVI))
print('The size of the data (first band):', NDVI.shape)
print('Minimum:', NDVI.min())
print('Maximum:', NDVI.max())
print('Mean:', NDVI.mean())
print('STD:', NDVI.std())
print('Data type of the array:',NDVI.dtype)

Vi kan også plotte NDVI-utregningen med `plt.imshow()`

In [ ]:
plt.imshow(NDVI,vmin=-0.3, vmax=1,cmap='RdBu')

## Videre lesning

Hvis du ønsker å lese mer om å jobbe med rasterdata i Python anbefaler jeg å ta en titt på:

- [Raster operasjoner i `Python for Geographic Data Analysis`](https://pythongis.org/part2/chapter-07/nb/02-common-raster-operations.html)
- [Kapittel 11, 12 og 13 i `Introduction to GIS Programming`](https://geog-312.gishub.org/book/geospatial/rasterio.html)